# Getting started with *voeventdb.remote*
In this notebook, we will cover the basics of working with voeventdb.remote, and give a quick tour of the sorts of data you can retrieve.

First, let's set up logging - by switching on 'DEBUG' level logging, we get to see the HTTP requests that are being made behind the scenes every time you run a query using voeventdb.remote. (NB we'll be using the Python-3 style print operator.)

In [ ]:
from __future__ import print_function
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

Next, library imports: Most of the user-facing code lives in ``voeventdb.remote.apiv1``, but we'll also alias ``voeventdb.remote`` to the shorthand ``vr``:

In [ ]:
import voeventdb.remote as vr
import voeventdb.remote.apiv1 as apiv1

----------
### Aside: Autocompletion tricks
For those new to IPython / notebooks: note you can use tab-completion to help you explore / save on typing. For example, type ``apiv1.`` in a code-cell and then hit **Tab** to see your available options. Likewise, position your cursor on a function and hit **Shift+Tab** to see a summary of function arguments, etc.

------------

There are a couple of variables living in the top-level ``voeventdb.remote`` module; for example the ``default_host``. You can set this to avoid specifying a host for every query. By default it's set to point at the [4PiSky](http://4pisky.org/voevent) server:

In [ ]:
# For testing against a local-dev server:
# vr.default_host = 'http://localhost:5000'
print(vr.default_host)

OK, let's query the database. We use different HTTP addresses, called 
[endpoints](http://voeventdb.4pisky.org/docs/apiv1/howto.html#finding-and-using-endpoints), to give us different summaries or subsets of the database.
We'll start off by using ``apiv1.count`` to find out how many packets are stored in this database - if it's empty, any other queries won't do us much good:

In [ ]:
apiv1.count()

Getting on for a million packets, that's quite lot! So there's plenty here for us to work with. 

## IVORN's and single-packet details

Next let's try ``apiv1.ivorn``, which returns a list of 
[IVORN's](http://voevent.readthedocs.org/en/latest/reading.html#ivorns-and-identifiers), unique idenitifiers which represent a VOEvent packet, analogous to how a URL represents a web-page.

In [ ]:
ivorn_list = apiv1.ivorn()
len(ivorn_list)

In [ ]:
#Take a look at the first 10
ivorn_list[:10]

We got back a list of IVORN's as expected - but only 5000 of them, not a million. By default, voeventdb.remote applies a limit to the maximum number of entries returned by a list query, as defined in ``default_list_n_max``:

In [ ]:
print(vr.default_list_n_max)

You can override this, or set it to ``0`` to simply fetch all matching results (though this can take a while).

In [ ]:
## How to override the default:
# vr.default_list_n_max = 2500
## Set n_max for a single query:
# apiv1.ivorn(n_max=50)

If we know the IVORN of a packet, we can look up some details using the ``apiv1.synopsis`` endpoint:

In [ ]:
ivorn = ivorn_list[0]
apiv1.synopsis(ivorn)

And we can even retrieve the raw XML, (i.e. the original packet), so we can pull out any data specific to a given observatory, etc:

In [ ]:
xml = apiv1.xml(ivorn)

A convenient way to inspect the xml-data is to use the [voevent-parse](http://voevent-parse.readthedocs.org/) library (which even has its very own [tutorial](https://github.com/timstaley/voevent-parse-tutorial)).

In [ ]:
## A brief example, see the voevent-parse tutorial for more
# import voeventparse as vp
# voe_tree = vp.loads(xml)
# vp.pull_params(voe_tree)

## Summary endpoints

So we can view individual packet IVORNS, and look up packet details, but that's very fine-grained information - we don't want to wade through all of them to figure out what's in the database. We can use 'summary' endpoints to give us a higher-level overview. 
For example, we can organise VOEvents by 'streams' - basically the bit of the IVORN between the `ivo://` prefix and the `#` symbol. The ``apiv1.stream_count`` endpoint tells us how many VOEvent packets belong to each stream:

In [ ]:
apiv1.stream_count()

Alternatively, we can slice up the packets by the authoring timestamp, binning by calendar month:

In [ ]:
apiv1.authored_month_count()

Or, we can divide them up by assigned [role](http://voevent.readthedocs.org/en/latest/reading.html#event-roles),
which comes in three flavours:

In [ ]:
apiv1.role_count()

Similarly, ``apiv1.stream_role_count`` gives a breakdown of how many packets belong to each role, listed by stream.

## Coming next ...
So now that we have an idea of what's in the database, how do we choose a category and drill-down to select specific VOEvents? For that we need filters, which are covered in tutorial 2.